## Load Packages

In [1]:
using Roots

include("printmat.jl")

printlnPs (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
    default(show=false)               #for pyplot: avoids pop-ups
else    
    gr(size=(600,400))
    default(show=true)
end

# Interest Rate vs (Zero Coupon) Bond Price

Recall: the bond price (for maturity $m$) and the the effective interest rate follow

$B(m) = [1+Y(m)]^{-m}$

Instead, with a continuously compounded interest rate we have 

$B(m) = e^{-m y(m)}$

In [3]:
Ym  = 0:0.01:0.07               #consider different interest rates
B1  = (1 + Ym).^(-1)
B10 = (1 + Ym).^(-10)


plot(Ym,B1,color=:red,linewidth=2,label="1y")
plot!(Ym,B10,color=:green,line=(:dash,2),label="10y")
title!("Bond prices and rates")
xlabel!("Interest rate")
ylabel!("Bond price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 0.04 
 
 
 0.05 
 
 
 0.06 
 
 
 0.07 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 Bond prices and rates 
 
 
 Interest rate 
 
 
 Bond price 
 
 
 
 
 
 
 
 1y 
 
 
 
 10y

In [4]:
m  = 0.5                              #maturity
B  = 0.95                             #bond price
Y  = B^(-1/m) - 1                     #effective interest rate
y  = -log(B)/m                        #continuously compounded interest rate

printlnPs("Effective and continuously compounded interest rates: ",[Y y])

Effective and continuously compounded interest rates:      0.108     0.103


# Bond Returns (Zero Coupon Bonds)

Let $B_t$ denote the price of a bond in period $t$ and $B_{t+s}$ the price of the same bond $s$ periods later. (For simplicity, we skip the notation for maturity.)

The log return of holding this bond is clearly $\ln (B_{t+s}/B_t)$

In [5]:
y₁ = 0.005                  #interest rate before
y₂ = 0.015                  #interest rate after  

B₁ = [exp(-1*y₁) exp(-10*y₁)]  #bond prices (1-year and 10 year) before
B₂ = [exp(-1*y₂) exp(-10*y₂)]  #bond prices (1-year and 10 year) after

R₂ = log.(B₂./B₁)               #returns

printlnPs("Bond return as bond prices change from ")
printlnPs(B₁)
printlnPs("to ")
printlnPs(B₂)

printlnPs("\n",R₂)

Bond return as bond prices change from 
     0.995     0.951
to 
     0.985     0.861

    -0.010    -0.100


# Forward Rates

From the forward-spot parity we have 

$F=\left[  1+Y(m)\right]  ^{m}B(n)=B(n)/B(m)$

and a foward interest rate can then be defined as 

$\Gamma(m,n)  =F^{-1/(n-m)}-1$

In [6]:
m = 0.5
n = 0.75
Ym = 0.04
Yn = 0.05
Bn = (1+Yn)^(-n)
F  = (1+Ym)^m*Bn

printlnPs("Price of $n-period bond and the foward price for delivery in period $m: ",[Bn F])

Γ = F^(-1/(n-m)) - 1
printlnPs("\nImplied forward rate: ",Γ)

Price of 0.75-period bond and the foward price for delivery in period 0.5:      0.964     0.983

Implied forward rate:      0.070


# Coupon Bond Prices

Recall, the coupon bond price $P$ is the present value of the future cash flows $cf_k$

$P = \sum_{k=1}^{K} \frac{cf_{k}}{\left[  1+Y(m_{k})\right]  ^{m_{k}}}$

and the yeld to maturity $\theta$ solves

$P = \sum_{k=1}^{K} \frac{cf_{k}}{(1+\theta)  ^{m_{k}}}$

The BondPrice3Ps(Y,cf,mk) function (see below) can handle both the case when $Y$ is a vector (like in the first
equation) and when Y is a scalar (like $\theta$ in the second equation).

In [7]:
function BondPrice3Ps(Y,cf,mk)              #cf is a vector of all cash flows
    cfac = cf./((1+Y).^mk)                  #c/(1+y1)^t1 + c/(1+y2)^t2 + ...+ c/(1+y2)^m
    P    = sum(cfac)                        #price
    return P
end

BondPrice3Ps (generic function with 1 method)

In [8]:
Y1 = 0.95^(-1) - 1                          #B(1) = 0.95 -> spot-rate, 1-year horizon
Y2 = 0.90^(-1/2) - 1                        #B(2) = 0.90 -> spot-rate, 2-year horizon 
printlnPs("Y1 and Y2: ",[Y1 Y2])

c = 0.06
P =  BondPrice3Ps([Y1;Y2],[c;c+1],[1;2])
printlnPs("\nPrice of 2y bond with $c coupon: ",P)

c = 0.09
P =  BondPrice3Ps([0.06;0.091],[c;c+1],[1;2])
printlnPs("\nPrice of a 2y par (almost) bond with $c coupon: ",P)

Y1 and Y2:      0.053     0.054

Price of 2y bond with 0.06 coupon:      1.011

Price of a 2y par (almost) bond with 0.09 coupon:      1.001


# Yield to Maturity

In [9]:
c = 0.04
Y = 0.03                   #all spot rates are 3%    
P = BondPrice3Ps([Y;Y],[c;c+1],[1;2])
ytm = fzero(y->BondPrice3Ps(y,[c;c+1],[1;2])-P,[-0.1;0.1])   #solving for ytm
printlnPs("Price of 2-year $c coupon bond: ",P)
printlnPs("\n2-year $c coupon bond, ytm: ",ytm)

c = 0.04
Y = 0.04                   #all spot rates are 4%    
P = BondPrice3Ps([Y;Y],[c;c+1],[1;2])
ytm = fzero(y->BondPrice3Ps(y,[c;c+1],[1;2])-P,[-0.1;0.1])
printlnPs("\n2-year $c coupon bond, ytm: ",ytm)

P = 1/1.07 + 1/1.1^3        #portfolio of B(1) and B(3)
ytm = fzero(y->BondPrice3Ps(y,[1;1],[1;3])-P,[-0.1;0.1])
printlnPs("\nportfolio of B(1) and B(3), ytm: ",ytm)

Price of 2-year 0.04 coupon bond:      1.019

2-year 0.04 coupon bond, ytm:      0.030

2-year 0.04 coupon bond, ytm:      0.040

portfolio of B(1) and B(3), ytm:      0.091


# The Return from Holding a Bond until Maturity

The portfolio holds a 2-period coupon bond until maturity. All coupons are reinvested at the forward rates (so the total return is know already in the initial period).

In [10]:
function BillPricePs(Y,m)             #trivial function
    B = (1+Y)^(-m)
    return B
end

BillPricePs (generic function with 1 method)

In [11]:
Y1₀ = 0.04                       #spot interest rate in period 0 for 1y bill
Y2₀ = 0.05             
B1₀ = BillPricePs(Y1₀,1)         #zero coupon bond prices for the same maturities
B2₀ = BillPricePs(Y2₀,2)         

c = 0.03
P2 =  BondPrice3Ps([Y1₀;Y2₀],[c;c+1],[1;2])
println("Prices of 1- and 2-year zero coupon bonds and a 2-year $c coupon bond (at t)")
printmat([B1₀ B2₀ P2])

Value₂ = c*B1₀/B2₀ + c + 1
R      = Value₂/P2
printlnPs("Portfolio value in t+2, coupons have been reinvested using forward contacts: ",Value₂)

printlnPs("\nTotal portfolio gross return from t to t+2: ",R,
        "\nCompare with (1+Y₀[2])^2: ",(1+Y2₀)^2)

Prices of 1- and 2-year zero coupon bonds and a 2-year 0.03 coupon bond (at t)
     0.962     0.907     0.963

Portfolio value in t+2, coupons have been reinvested using forward contacts:      1.062

Total portfolio gross return from t to t+2:      1.103
Compare with (1+Y₀[2])^2:      1.103


# The Return from Selling a Bond before Maturity

The portfolio holds a 2-period coupon bond for one period (from $t$ to $t+1$). All coupons are reinvested at the forward rates in the same way as if we were holding the bond until maturity. This whole portfolio is sold in $t+1$. At that time, the value is the known value at $t+2 \times B_1(1)$, since this is the PV(known value at $t+2$)

## Case 1: Unchanged Interest Rates

In [12]:
println("The interest rates and bond prices in period 0 are the same as in the previous example")

Y1₁ = 0.04                      #1-year interest rate in period 1 (unchanged)
B1₁ = BillPricePs(Y1₁,1)        #bill price in period 1    

Value₁ = Value₂*B1₁
R      = Value₁/P2

printlnPs("\nPortfolio value in t+1, coupons have been reinvested using forward contacts: ",Value₁)
printlnPs("\nTotal portfolio return from t to t+1: ",R,
"\nCompare with B1₁/B2₀: ",B1₁/B2₀)

The interest rates and bond prices in period 0 are the same as in the previous example

Portfolio value in t+1, coupons have been reinvested using forward contacts:      1.021

Total portfolio return from t to t+1:      1.060
Compare with B1₁/B2₀:      1.060


## Case 2: Higher Interest Rates

In [13]:
println("Once again, the interest rates and bond prices in period 0 are the same as in the previous example")

Y1₁ = 0.045                      #1-year interest rate in period 1 (unchanged)
B1₁ = BillPricePs(Y1₁,1)        #bill price in period 1    

Value₁ = Value₂*B1₁
R      = Value₁/P2

printlnPs("\nPortfolio value in t+1, coupons have been reinvested using forward contacts: ",Value₁)
printlnPs("\nTotal portfolio return from t to t+1: ",R,
"\nCompare with B1₁/B2₀: ",B1₁/B2₀)

Once again, the interest rates and bond prices in period 0 are the same as in the previous example

Portfolio value in t+1, coupons have been reinvested using forward contacts:      1.016

Total portfolio return from t to t+1:      1.055
Compare with B1₁/B2₀:      1.055


# Bootstrapping

Recall: with information about coupons $c(k)$ and coupon bond price $P(m)$, we solve for the implied zero coupon bond prices $B(s)$ from

$
\begin{bmatrix}
P(1)\\
P(2)\\
P(3)
\end{bmatrix} =
\begin{bmatrix}
c(1)+1 & 0 & 0\\
c(2) & c(2)+1 & 0\\
c(3) & c(3) & c(3)+1
\end{bmatrix}
\begin{bmatrix}
B(1)\\
B(2)\\
B(3)
\end{bmatrix}
$

In [14]:
c  = [0;0.06]                    #coupon rates of the bonds (here 2)
P  = [0.95;1.01]                  #coupon bond prices
mk = [1;2]  

n = length(c)                           
cfMat = eye(n)
for i = 1:n                     #fill the cfMat matrix 
    for j = 1:i
        cfMat[i,j] = cfMat[i,j] + c[i]
    end
end    

println("The cash flow matrix")
printmat(cfMat)

println("Solve P = cfMat*B for B (implied zero-coupon bond prices):")
B = cfMat\P
printmat(B)

Y = zeros(n)                   #solve for the implied spot rates
for i = 1:n
    Y[i] = B[i]^(-1/mk[i]) - 1
end    
println("Implied spot interest rates:")
printmat(Y)

The cash flow matrix
     1.000     0.000
     0.060     1.060

Solve P = cfMat*B for B (implied zero-coupon bond prices):
     0.950
     0.899

Implied spot interest rates:
     0.053
     0.055



# Estimating Yield Curve with Regression Analysis

Recall: with a quadratic discount function

$B(m)=a_{0}+a_{1}m+a_{2}m^{2}$,

we can write the coupon bond price 

$P\left(  m_{K}\right)   =\sum_{k=1}^{K}B(m_{k})c+B(m_{K})$
as 

$P\left(  m_{K}\right)   =\sum_{k=1}^{K}\left(  a_{0}+a_{1}m_{k}+a_{2}m_{k}^{2}\right)  c+\left(
a_{0}+a_{1}m_{K}+a_{2}m_{K}^{2}\right)$

Collect terms as 

$
P\left(  m_{K}\right)  =a_{0}\underset{\text{term 0}}{\underbrace{(Kc+1)}
}+a_{1}\underset{\text{term 1}}{\underbrace{(c
{\textstyle\sum_{k=1}^{K}}
m_{k}+m_{K})}}+a_{2}\underset{\text{term 2}}{\underbrace{(c
{\textstyle\sum_{k=1}^{K}}
m_{k}^{2}+m_{K}^{2})}}$

We estimate $(a_0,a_1,a_2)$ by using the "terms" as regressors.


In [15]:
x = zeros(n,3)                      #create regressors for quadratic model: 3 columns
for i = 1:n                         #x[i,j] is for bond i, regressor j
    x[i,1] = mk[i]*c[i] + 1
    x[i,2] = c[i]*sum(1:mk[i]) + mk[i]
    x[i,3] = c[i]*sum((1:mk[i]).^2) + mk[i]^2
end    

println("regressors")
printmat(x)

a = x\P                           #regress Bc on x
println("regression coefficients in discount fn")
printmat(a)

m = 1:5
B = a[1] + a[2]*m + a[3]*m.^2      #fitted discount function
println("maturities and fitted discount fn")
printmat([m B])

print("Check that m=1 and m=2 give the same results as in the bootstrap (as many obs as coeffs in this case)")

regressors
     1.000     1.000     1.000
     1.120     2.180     4.300

regression coefficients in discount fn
     0.693
     0.411
    -0.154

maturities and fitted discount fn
     1.000     0.950
     2.000     0.899
     3.000     0.540
     4.000    -0.126
     5.000    -1.101

Check that m=1 and m=2 give the same results as in the bootstrap (as many obs as coeffs in this case)